# Proyecto Final

***Equipo 07***

- Aide Jazmín González Cruz
- Elena Villalobos Nolasco
- Carolina Acosta Tovany

#### Instrucciones

El proyecto/examen final consistirá en:

La implementación del algoritmo de filtrado colaborativo utilizando la metodología vista en clase (uso de otra metodología no se calificará).

Todos los algoritmos de aprendizaje de máquina que se utilicen deberán haber sido creados por ustedes. Sólo podrán utilizar Transformers y funciones de apoyo de scikit-learn (para realizar la división de los datos en entrenamiento y prueba, o el procedimiento de validación cruzada, etc.) mas ningún estimator (regresión logística, máquina de vectores de soporte, k medias, etc.). 

Se deberá explicar como se obtuvo la k con la que se generó el resultado final.

Se utilizarán los archivos con el conjunto pequeño de calificaciones y películas ubicado en la siguiente https://www.kaggle.com/rounakbanik/the-movies-dataset:

- **links_small.csv**: Contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset.

- **ratings_small.csv**: The subset of 100,000 ratings from 700 users on 9,000 movies.

Con el fin de mejorar la calificación (opcional, puntos extra), se podrán utilizar los algoritmos desarrollado en las tareas del curso y los datos relevantes (los que hacen match con los datos anteriores) contenidos en los archivos:

- **movies_metadata.csv**: The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

- **keywords.csv**: Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.

- **credits.csv**: Consists of Cast and Crew Information for all our movies. Available in the form of a stringified JSON Object.

La métrica con la que se determinará el desempeño del algoritmo es el NDCG 

(https://en.wikipedia.org/wiki/Discounted_cumulative_gain#Normalized_DCG)

Una vez obtenida la matriz de calificaciones, el programa deberá ser capaz de regresar las 5 mejores recomendaciones del o de los usuarios que se consulten.

El proyecto se entregará en un Jupyter notebook. El readme file debe contener las instrucciones para que se ejecute el código. Deben cerciorarse que siguiendo esas instrucciones el programa corre sin errores. 

Se deberá subir a la carpeta proyecto_final/equipo_xx en el repositorio GitHub antes de las 7:00 am del día del examen final (14 de diciembre de 2020).    

In [3]:
# Importación de paqueterías necesarias
import pandas as pd
import numpy as np
import random

La función objetivo:
    
$$J(X) = \frac{1}{2} \displaystyle\sum_{(a,i)\in\mathbb{D}} \left(Y_{ai}-\left [ UV^T \right ]_{ai} \right)^2 + \frac{\lambda}{2} \displaystyle\sum_{a=1}^n \displaystyle\sum_{j=1}^k U_{aj}^2 + \frac{\lambda}{2} \displaystyle\sum_{i=1}^m \displaystyle\sum_{j=1}^k V_{ij}^2$$

<span style="color:red; font-size: xx-large; font-weight: bold;">Cambio está función</span>

In [83]:
def load_data():
    """
    Carga datos
    Regresa 3 datasets:
    - Matriz de suarios, Películas y raitings. 
    - Arreglo de id de péliculas y nombres
    - Arreglo de nombres (id_pelicula)
    """
    
    # ----------- Carga de datos para la matriz de raitings -------------------
    links_small = pd.read_csv('links_small.csv')
    ratings_small = pd.read_csv('ratings_small.csv')
    
    # Películas en catálogo que no han calificado los usuarios
    df_mov_u = pd.DataFrame(ratings_small['movieId'])
    df_mov = pd.DataFrame(links_small['movieId'])

    common = df_mov.merge(df_mov_u, on=["movieId"])
    result = df_mov[~df_mov.movieId.isin(common.movieId)]
    
    # Construyendo la matriz Y_ai
    y_ia = links_small.set_index('movieId').join(ratings_small.set_index('movieId'))
    y_ia = y_ia.reset_index()
    #y_ia.pivot(index="userId", columns="movieId", values="rating") 
    y_ia = pd.DataFrame(y_ia.pivot(index='userId', columns='movieId', values='rating'))
    y_ia = pd.DataFrame(y_ia.to_records())
    # Eliminando usuario Nan
    y_ia = y_ia[pd.notnull(y_ia['userId'])]
    # Borrando columna 1 con user_id
    y_ia = y_ia.drop(['userId'], axis=1)
    # Cambiando Nan por zeros
    #y_ia[np.isnan(y_ia)] = 0
    
    # -------------------- Catálogo de películas ---------------------------
    # Carga de datos
    movies_metadata = pd.read_csv('movies_metadata.csv',low_memory=False)
    # Procesamiento del archivo
    hola1 = pd.DataFrame(movies_metadata[['id','title']])
    
    df_mov_u = pd.DataFrame(ratings_small['movieId'])
    df_mov = pd.DataFrame(links_small[['tmdbId','movieId']])
    
    hola1 = hola1.rename(columns = {'id':'tmdbId'})
    
    malos = hola1[hola1['tmdbId'].str.contains('-')].index
    
    hola1 = hola1.drop(index=malos)
    hola1['tmdbId']=hola1['tmdbId'].apply(lambda x: float(x))
    
    df_mov=df_mov.dropna(0)
    id_base=df_mov.sort_values(by=['tmdbId'])
    titles_id= hola1.sort_values(by=['tmdbId'])
    bases_nombres_id=id_base.merge(titles_id, on=['tmdbId'],how='left')
    
    # -------------------- Catálogo id películas ---------------------------
    id_movies = y_ia.columns.tolist()
    id_movies = np.array(id_movies).astype(int)
    
    return y_ia, bases_nombres_id,id_movies
    

<span style="color:red; font-size: xx-large; font-weight: bold;">Por tanto devuelve 3 elementos</span>

In [84]:
Y, bases_nombres_id, arr_movies = load_data()
Y

,1,2,3,4,5,6,7,8,9,10,...,161830,161918,161944,162376,162542,162672,163056,163949,164977,164979
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
670,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
bases_nombres_id

,tmdbId,movieId,title
0,2.0,4470,Ariel
1,5.0,18,Four Rooms
2,6.0,479,Judgment Night
3,11.0,260,Star Wars
4,12.0,6377,Finding Nemo
...,...,...,...
9124,402672.0,162672,Mohenjo Daro
9125,410612.0,162376,NaN
9126,410803.0,164979,NaN
9127,410921.0,875,Nothing to Lose


In [85]:
arr_movies

array([     1,      2,      3, ..., 163949, 164977, 164979])

In [63]:
Y.shape

(671, 9125)

In [37]:
maxValues = Y.max() 
maxValues = pd.DataFrame(maxValues)
maxValues = maxValues.max() 
maxValues

0    5.0
dtype: float64

In [39]:
minValues = Y.min() 
minValues = pd.DataFrame(minValues)
minValues = minValues.min() 
minValues

0    0.5
dtype: float64

In [6]:
# Cambiando Nan por zeros
Y_0 = Y.copy()
Y_0[np.isnan(Y_0)] = 0
Y_0

,1,2,3,4,5,6,7,8,9,10,...,161830,161918,161944,162376,162542,162672,163056,163949,164977,164979
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
670,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<span style="color:blue; font-size: xx-large; font-weight: bold;">Prueba de la función</span>

In [105]:
Prueba = Y_0.iloc[[0,1,2,3,4], : 10]
Prueba

,1,2,3,4,5,6,7,8,9,10
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
# gregando más datos de prueba
Prueba.iloc[1][1] = 1
Prueba.iloc[1][4] = 5
Prueba

,1,2,3,4,5,6,7,8,9,10
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,4.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
a_m = arr_movies[:10]
a_m

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [107]:
# Fijando semilla
random.seed(0)
random_matrix = np.random.randint(1,5,(5,10))
random_matrix = pd.DataFrame(random_matrix)
# Agregando calificaciones de la matriz original
random_matrix.iloc[1][1] = 1
random_matrix.iloc[1][4] = 5
random_matrix.iloc[1][9] = 4
# Iniciando rows en 1
random_matrix.index = np.arange(1,len(random_matrix)+1)
# Igual para las columnas
random_matrix.columns += 1
random_matrix

,1,2,3,4,5,6,7,8,9,10
1,2,1,4,2,3,2,2,4,2,1
2,2,1,1,3,5,4,4,2,1,4
3,4,4,3,2,2,2,3,1,4,1
4,1,2,4,1,3,4,4,4,4,1
5,2,4,2,3,1,3,4,3,3,3


<span style="color:red; font-size: xx-large; font-weight: bold;">Se añadio el nombre de la película que hizo Elena y recibe arreglos por usuario dado</span>

In [134]:
def recomendaciones_id(Y, MCompleta, array_movies, bases_nombres_id, user, top=5):
    """
    Regresa id de las peliculas reomendadas dado un usuario
    Param: Recibe matriz con Nan, id del usurario y el top por default 5
    Return: Arreglo de ids de las películas sugeridas
    """
    
    # ------------------------------------ Para nuevas recomendaciones------------------------------
    # Se multiplica boleano con Y para poner en cero los ratings dados por el usuario
    nvos = []
    # Agregando índice
    for i in range (len(MCompleta)):
        # Se concatena el indice de la pélicula
        nvos.append([array_movies[i],MCompleta[i]*(~Y[i].any())])
    
    # nvos es un arreglo se pasa a dataframe para mejor manejabilidad
    nvos = pd.DataFrame(nvos)
    # se colocan nombres a las columnas
    nvos.columns = ['id_movie','rating_recom']
    # se ordenan de forma descendente
    nvos = nvos.sort_values(by=['rating_recom'], ascending=False)
    # Borrando 0
    nvos = nvos[(nvos[['rating_recom']] != 0).all(axis=1)]
    # Obteniendo el top
    recomendaciones = nvos['id_movie'].head(5).to_numpy()
    
    # ------------------------------------ Para recomendaciones incluyendo las existentes ----------
    # Incluyendo los datos calificados
    todos = []
    # Agregando índice
    for i in range (len(MCompleta)):
        # Se concatena el indice de la pélicula
        todos.append([array_movies[i], MCompleta[i]])
    # todos es un arreglo se pasa a dataframe para mejor manejabilidad
    todos = pd.DataFrame(todos)
    # se colocan nombres a las columnas
    todos.columns = ['id_movie','rating_recom']
    # se ordenan de forma descendente
    todos = todos.sort_values(by=['rating_recom'], ascending=False)
    # Obteniendo el top
    recomendaciones_t = todos['id_movie'].head(5).to_numpy()  
    
    # Por si se quiere imprimir desde aqui las recomendaciones
    print("Estás son las nuevas películas que te recomendamos usuario no. ", user)
    print(bases_nombres_id[bases_nombres_id['movieId'].isin(rec_nvas)][["movieId", "title"]])
    
    print("Estás películas te pueden interesar usuario no. ", user)
    print(bases_nombres_id[bases_nombres_id['movieId'].isin(rec_todas)][["movieId", "title"]])
    
    return recomendaciones, recomendaciones_t


In [139]:
id_user = 2
#Prueba.loc[user,:]

In [140]:
# random_matrix.loc[user,:]

In [141]:
# Prueba es la matriz Y con 0 en NaN y random_matrix es la matriz predicha

# Para matriz predicha como array descomentar***
# recomendaciones_id(np.array(Prueba.loc[user,:]), np.array(random_matrix.iloc[user-1]))

# Para dataframes***
#recomendaciones_id(np.array(Prueba.loc[user,:]), np.array(random_matrix.loc[user,:]))

# Para ambos array
Prueba = np.array(Prueba)
random_matrix = np.array(random_matrix)

rec_nvas, rec_todas = recomendaciones_id(np.array(Prueba)[id_user-1], np.array(random_matrix)[id_user-1], 
                                         a_m, bases_nombres_id, id_user)


Estás son las nuevas películas que te recomendamos usuario no.  2
      movieId              title
603         1          Toy Story
671         6               Heat
3639        7            Sabrina
6253        4  Waiting to Exhale
7282        8       Tom and Huck
Estás películas te pueden interesar usuario no.  2
      movieId                        title
501        10                    GoldenEye
671         6                         Heat
3639        7                      Sabrina
3641        5  Father of the Bride Part II
6253        4            Waiting to Exhale


<span style="color:red; font-size: xx-large; font-weight: bold;">Sin embargo si no se quiere así, como regresa los arreglos se puede hacer afuera</span>

In [132]:
print("Estás son las nuevas películas que te recomendamos usuario no. ", user)
bases_nombres_id[bases_nombres_id['movieId'].isin(rec_nvas)][["movieId", "title"]]

Estás son las nuevas películas que te recomendamos usuario no.  2


,movieId,title
603,1,Toy Story
671,6,Heat
3639,7,Sabrina
6253,4,Waiting to Exhale
7282,8,Tom and Huck


In [133]:
print("Estás películas te pueden interesar usuario no. ", user)
bases_nombres_id[bases_nombres_id['movieId'].isin(rec_todas)][["movieId", "title"]]

Estás películas te pueden interesar usuario no.  2


,movieId,title
501,10,GoldenEye
671,6,Heat
3639,7,Sabrina
3641,5,Father of the Bride Part II
6253,4,Waiting to Exhale
